[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/akwasnie/vehicle_sensing/blob/detection_and_distance/object_tracking_and_dist_calculation.ipynb)

In [ ]:
# In Jupyter, you would need to install TF 2 via !pip.
%tensorflow_version 2.x

In [ ]:
!pip install onnxruntime
!pip install range-key-dict

In [ ]:
import json
import os
import sys
from base64 import b64decode, b64encode

import cv2
import IPython
import numpy as np
import onnxruntime
import torch
from google.colab import output
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import Image, Javascript, clear_output, display
from numpy import asarray
from PIL import Image as pimage

In [ ]:
!git clone https://github.com/akwasnie/CenterNet.git && cd CenterNet && git checkout 8ef87b433529ac8f8bd4f95707f6bc05052c55e9

In [ ]:
sys.path.append('CenterNet')
from CenterNet import src
from src.lib.utils.image import get_affine_transform
from src.lib.models.decode import ctdet_decode
from src.lib.utils.post_process import ctdet_post_process

In [ ]:
DATA_DIR = 'data/'
MODEL_FILE = os.path.join(DATA_DIR, 'ctdet_ebike.onnx')
CLASS_FILE = os.path.join(DATA_DIR, 'ebike.json')
IMAGES_ZIP = os.path.join(DATA_DIR, 'distance_images.zip')
os.makedirs(DATA_DIR, exist_ok=True)

if not os.path.exists(MODEL_FILE):
    !gdown --id 168n5dPHGBoRTpZyZaH6V04AR5w-HYbC_ -O $MODEL_FILE
else:
    print('Model file ({}) already exists.'.format(MODEL_FILE))

if not os.path.exists(CLASS_FILE):
    !gdown --id 1qCIeX44a5zBaKpDa6KDz30H9qCULwgP2 -O $CLASS_FILE
else:
    print('Class file ({}) already exists.'.format(CLASS_FILE))
    
if not os.path.exists(IMAGES_ZIP):
    !gdown --id 1GlaTbuNagGgB-u1gzuJu3ANJ7kfzWeX9 -O $IMAGES_ZIP
    !unzip $IMAGES_ZIP
else:
    print('Images zip ({}) already exists.'.format(IMAGES_ZIP))

In [ ]:
IN_SIZE = [512,512]
MEAN = [0.408, 0.447, 0.470]
STD = [0.289, 0.274, 0.278]
MAX_BB_NUM = 1
THRESHOLD = 0.5
size_mapping = {
    'car': 200,
    'truck': 245,
    'bike': 65,
    'motorbike': 90,
    'bus': 255,
  }
focal_length_pixels = 890
#  <5m; <10m; <20m; <30m; <40m; <50m; <75m; <100m; <125m; <150m; <200m; <300m; <500m
from range_key_dict import RangeKeyDict
range_mapping = RangeKeyDict({
                (0, 500) : "<5m",
                (501, 1000) : "<10m",
                (1001, 2000) : "<20m",
                (2001, 3000) : "<30m",
                (3001, 4000) : "<40m",
                (4001, 5000) : "<50m",
                (5001, 7500) : "<75m",
                (7501, 10000) : "<100m",
                (10001, 12500) : "<125m",
                (12501, 15000) : "<150m",
                })

In [ ]:
def preprocess(image_path):
  image = cv2.imread(image_path)
  frame = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  h, w = frame.shape[:2]
  c = [np.array([w/2., h/2.], dtype=np.float32)]
  s = [max(h, w) * 1.0]
  trans_input = get_affine_transform(c[0], s[0], 0, IN_SIZE)
  im_data = cv2.warpAffine(
      frame, trans_input,
      (IN_SIZE[0], IN_SIZE[1]),
      flags=cv2.INTER_LINEAR,
  )
  im_data = ((im_data / 255. - MEAN) / STD).astype(np.float32)
  im_data = im_data.transpose(2, 0, 1).reshape(1, 3, IN_SIZE[0], IN_SIZE[1])
  return frame, im_data, c, s

def import_data(model_path, class_mapping_path):
  ort_session = onnxruntime.InferenceSession(model_path)

  with open(class_mapping_path, 'r') as fp:
    category_index = json.load(fp)
  return ort_session, category_index

def centernet2cocodict(out, c, m, category_index):
    hm = torch.from_numpy(out[0])
    wh = torch.from_numpy(out[1])
    reg = torch.from_numpy(out[2])
    hm = hm.sigmoid_()
    dets = ctdet_decode(hm, wh, reg=reg, cat_spec_wh=False, K=MAX_BB_NUM)
    dets = dets.detach().cpu().numpy()
    dets = ctdet_post_process(dets.copy(), c, m, 128, 128, 80)[0]

    results = {}  
    for j in range(1, 5 + 1):
        results[j] = np.array(dets[j], dtype=np.float32).reshape(-1, 5)
    
    scores = np.hstack([results[j][:, 4] for j in range(1, 5 + 1)])

    out_dict = {
        'num_detections': MAX_BB_NUM,
        'detection_boxes': [],
        'detection_scores': [],
        'detection_classes': [],
        }

    cats = list(category_index.values())
    cats.sort(key=lambda x: x['id'])

    for j in range(1, 5 + 1):
        for bbox in results[j]:
            if bbox[4] > THRESHOLD:
                out_dict['detection_boxes'].append(bbox[:4])
                out_dict['detection_scores'].append(bbox[4])
                out_dict['detection_classes'].append(cats[j]['id'])

    out_dict['detection_boxes'] = np.asarray(out_dict['detection_boxes'])
    out_dict['detection_scores'] = np.asarray(out_dict['detection_scores'])
    out_dict['detection_classes'] = np.asarray(out_dict['detection_classes'])

    return out_dict

def run_inference(ort_session, category_index, image_data, c, s):
  output = ort_session.run(None, {ort_session.get_inputs()[0].name: image_data})
  output_dict = centernet2cocodict(output, c, s, category_index)
  return output_dict

def calculate_distance(delta_x_pixels, class_name):
  distance = size_mapping[class_name] * focal_length_pixels/delta_x_pixels
  return range_mapping[int(distance)]

def draw_text(img, text,
          pos=(0, 0),
          font=cv2.FONT_HERSHEY_PLAIN,
          font_scale=5,
          font_thickness=2,
          text_color=(255, 0, 0),
          text_color_bg=(255, 255, 255, 0.5)
          ):

    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, (x-5, y-5), (x + text_w+5, y + text_h+5), text_color_bg, -1)
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)

    return text_size

ort_session, category_index = import_data(MODEL_FILE, CLASS_FILE)

In [ ]:
FILENAME = "distance_images/10_f.JPG"
frame, image_data, c, s = preprocess(FILENAME)
ort_session, category_index = import_data(MODEL_FILE, CLASS_FILE)
output_dict = run_inference(ort_session, category_index, image_data, c, s)
print(output_dict)
for i in range(len(output_dict['detection_boxes'])):
  bbox = output_dict['detection_boxes'][i]
  class_name = category_index[str(output_dict['detection_classes'][i])]['name']
  distance = calculate_distance(abs(int(bbox[2])-int(bbox[0])), class_name)
   # if width is greater than height, ignore - side view
  if (abs(int(bbox[2])-int(bbox[0]))>abs(int(bbox[3])-int(bbox[1]))):
    distance = ""
  print(distance)
  cv2.rectangle(frame,
                (int(bbox[0]), int(bbox[1])),
                (int(bbox[2]), int(bbox[3])),
                (255,0,0),
                2)
  draw_text(frame,
            class_name + " " + distance,
            (int(bbox[0]), int(bbox[1]-20)))
result = np.asarray(frame)
result = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
cv2_imshow(result)